# Parallel computing

### Start and check multiple processes

In [1]:
using BenchmarkTools

In [2]:
addprocs(2) # or start julia in a console with the option -p n_processes
# this start multiple processes not threads (processes doesn't share memory as threads does, but threads
# are limited to a single CPU)

2-element Array{Int64,1}:
 2
 3

In [3]:
for pid in workers()
    println(pid)
end
    

2
3


In [4]:
@everywhere println(myid())

1
	From worker 2:	2
	From worker 3:	3


### Parallel for -  @parallel for macro
- output_var = @parallel (aggregator) for_cycle 

In [2]:
function f(n)
  s = 0.0
  for i = 1:n
    s += i/2
  end
    return s
end

f (generic function with 1 method)

In [15]:
@time f(100000000)

  0.089326 seconds (1.50 k allocations: 69.067 KB)


2.500000025e15

In [7]:
function pf(n)
    s = @parallel (+) for i = 1:n # aggregate using sum on variable s 
        i/2                       # last element of for cycle is used by the aggregator
  end
  return s
end

pf (generic function with 1 method)

In [11]:
@time pf(100000000)

  0.048390 seconds (356 allocations: 27.766 KB)


2.500000025e15

### Parallel map - pmap(f,collection)
 - applies a function f on each elements of the collection

In [3]:
x = [rand(100,100) for i in 1:10];

In [17]:
@benchmark map(svd,x)

BenchmarkTools.Trial: 
  memory estimate:  5.47 MiB
  allocs estimate:  242
  --------------
  minimum time:     26.183 ms (0.00% GC)
  median time:      26.451 ms (0.00% GC)
  mean time:        26.588 ms (0.52% GC)
  maximum time:     28.261 ms (0.00% GC)
  --------------
  samples:          188
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [18]:
@benchmark pmap(svd,x)

BenchmarkTools.Trial: 
  memory estimate:  1.62 MiB
  allocs estimate:  1885
  --------------
  minimum time:     15.146 ms (0.00% GC)
  median time:      15.802 ms (0.00% GC)
  mean time:        16.158 ms (0.57% GC)
  maximum time:     25.246 ms (0.00% GC)
  --------------
  samples:          310
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%